<a href="https://colab.research.google.com/github/jeanlucjackson/w266_final_project/blob/main/data/datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring Datasets

## Downloading Datasets
Download using HuggingFace's `datasets` library

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Uncomment if `datasets` not installed
! pip install datasets

In [ ]:
from datasets import load_dataset

### SQuAD 2.0

In [ ]:
# SQuAD 2.0 is 40 MB and quick to download
data_squad = load_dataset("squad_v2")

⚠️ WARNING ⚠️ — TriviaQA is a large dataset

### TriviaQA*

In [ ]:
# TriviaQA downloads and pre-processes to generate 17.4 GB -- it takes 25 mins total
data_trivia = load_dataset("trivia_qa", "rc")

### Natural Questions**

In [ ]:
# Natural Questions is too big for memory at 138 GB
# data_nq = load_dataset("natural_questions")

### QuAC

In [ ]:
# Question Answering in Context downloads 136 MB and is quick
data_quac = load_dataset("quac")

## Getting Familiar

### SQuAD 2.0

In [ ]:
data_squad

In [ ]:
data_squad['train'].info.features

In [ ]:
# Look at first example (queen bey)
data_squad['train'][0]

### TriviaQA

In [ ]:
data_trivia

In [ ]:
data_trivia['train'].info.features

In [ ]:
# Look at first example
data_trivia['train'][5]

### Natural Questions

### QuAC

In [ ]:
data_quac

In [ ]:
data_quac['train']

In [ ]:
# Look at first example
data_quac['train'][12]